In [79]:
#IMPORTANDO BIBLIOTECAS
import numpy as np
import pandas as pd
import random

In [80]:
# INTERVALOS DE VALORES PARA OS CAMPOS
loa_min = 15
loa_max = 60

boca_min = 6
boca_max = 18

dwt_min = 30000
dwt_max = 200000

calado_entrada_min = 8
calado_entrada_max = 20

calado_saida_min = 8
calado_saida_max = 20

calado_aereo_min = 10
calado_aereo_max = 35

pontal_min = 5
pontal_max = 20

tamanho_lanca_min = 10
tamanho_lanca_max = 120

ano_construcao_min = 1998
ano_construcao_max = 2023

tipos_navio = ['GS', 'CG', 'GSM', 'GL', 'GLP', 'CNTR']


In [95]:
# NUMERO DE AMOSTRAS QUE SERÃO GERADAS
num_amostras = 2000

# GERAÇÃO DE VALORES ALEATÓRIOS VIA NUMPY
np.random.seed(0)
random.seed(0)

nomes = np.array(['Navio' + str(i) for i in range(1, num_amostras + 1)])

loa = np.random.randint(loa_min, loa_max, num_amostras)

boca = np.random.randint(boca_min, boca_max, num_amostras)

dwt = np.random.randint(dwt_min, dwt_max, num_amostras)

calado_entrada = np.random.randint(calado_entrada_min, calado_entrada_max, num_amostras)

calado_saida = np.random.randint(calado_saida_min, calado_saida_max, num_amostras)

calado_aereo = np.random.randint(calado_aereo_min, calado_aereo_max, num_amostras)

pontal = np.random.randint(pontal_min, pontal_max, num_amostras)

tamanho_lanca = np.random.randint(tamanho_lanca_min, tamanho_lanca_max, num_amostras)

ano_construcao = np.random.randint(ano_construcao_min, ano_construcao_max, num_amostras)

ultimo_porto = np.array(['Porto' + str(i) for i in np.random.randint(1, 6, num_amostras)])

proximo_porto = np.array(['Porto' + str(i) for i in np.random.randint(1, 6, num_amostras)])

tipo_navio = [random.choice(tipos_navio) for _ in range(num_amostras)]



In [96]:
#CRIAÇÃO DE DATAFRAME COM VALORES
data = {
    'Nome': nomes,
    'LOA (m)': loa,
    'Boca (m)': boca,
    'DWT (ton)': dwt,
    'Calado de Entrada (m)': calado_entrada,
    'Calado de Saída (m)': calado_saida,
    'Calado Aéreo (m)': calado_aereo,
    'Pontal (m)': pontal,
    'Tamanho de Lança (m)': tamanho_lanca,
    'Ano de Construção': ano_construcao,
    'Tipo do Navio': tipo_navio,
    'Último Porto': ultimo_porto,
    'Próximo Porto': proximo_porto

}

df = pd.DataFrame(data)

In [97]:
#CRIACAO DE CONDICOES PARA DETERMINAÇÃO DA CLASSE DE ACORDO COM PREVISORES
def determinar_classificacao(df):
    condicao_loa = df['LOA (m)'] <= 280
    condicao_boca = df['Boca (m)'] <= 50
    condicao_dwt = df['DWT (ton)'] <= 155000
    condicao_ce = df['Calado de Entrada (m)'] <= 18.5
    condicao_cs = df['Calado de Saída (m)'] <= 18.5
    condicao_idade = 2023 - df['Ano de Construção'] < 18

    combinacao = condicao_loa and condicao_boca and condicao_dwt and condicao_ce and condicao_cs and condicao_idade
    return 1 if combinacao else 0

In [98]:
#CRIACAO DA CLASSE 'SITUACAO' 0 - NEGADO, 1 - APROVADO
df['Situação'] = df.apply(determinar_classificacao, axis=1)
df

,Nome,LOA (m),Boca (m),DWT (ton),Calado de Entrada (m),Calado de Saída (m),Calado Aéreo (m),Pontal (m),Tamanho de Lança (m),Ano de Construção,Tipo do Navio,Último Porto,Próximo Porto,Situação
0,Navio1,59,12,93953,11,8,21,14,16,2019,GL,Porto3,Porto5,1
1,Navio2,15,11,144548,9,8,25,6,90,2003,GL,Porto2,Porto5,0
2,Navio3,18,7,108851,11,18,20,19,42,2018,GS,Porto1,Porto3,1
3,Navio4,18,16,111313,16,14,31,9,44,1999,GSM,Porto4,Porto5,0
4,Navio5,54,11,47921,11,10,21,12,20,2016,GLP,Porto5,Porto4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,Navio1996,31,16,61455,12,12,26,9,89,2006,GS,Porto2,Porto2,1
1996,Navio1997,26,11,30919,11,18,34,5,53,2012,CNTR,Porto5,Porto3,1
1997,Navio1998,26,7,148026,19,9,29,6,53,2020,GL,Porto1,Porto3,0
1998,Navio1999,59,11,187581,11,12,29,7,51,2011,GSM,Porto4,Porto2,0


In [99]:
# VERIFICAÇÃO DO BALANCEAMENTO DA CLASSE
df['Situação'].value_counts()

Situação
0    1172
1     828
Name: count, dtype: int64

In [103]:
# CRIAÇÃOD DE INSTÂNCIAS SEPARADAS DAS DUAS AMOSTRAS
df_aceitos = df[df['Situação'] == 1].head(300).copy()
df_recusados = df[df['Situação'] == 0].head(60).copy()

60

In [106]:
# CRIAÇÃO DE DATAFRAME COM MAIS ACEITOS QUE RECUSADOS PARA TESTAGEM POSTERIOR
df_misto = pd.concat([df_aceitos, df_recusados], ignore_index=True)
df_misto = df_misto.sample(frac=1).reset_index(drop=True)

,Nome,LOA (m),Boca (m),DWT (ton),Calado de Entrada (m),Calado de Saída (m),Calado Aéreo (m),Pontal (m),Tamanho de Lança (m),Ano de Construção,Tipo do Navio,Último Porto,Próximo Porto,Situação
0,Navio46,50,14,140777,17,15,24,12,44,2014,GS,Porto2,Porto4,1
1,Navio610,16,17,94901,13,12,32,13,63,2020,GSM,Porto4,Porto5,1
2,Navio470,22,7,154178,17,11,25,7,12,2009,GLP,Porto5,Porto4,1
3,Navio480,46,10,62903,10,18,25,19,63,2013,CG,Porto5,Porto3,1
4,Navio197,39,16,89097,16,16,30,10,111,2019,GSM,Porto3,Porto1,1
5,Navio341,34,17,133313,13,9,30,7,68,2018,GSM,Porto5,Porto2,1
6,Navio124,42,9,89827,16,8,31,16,101,2019,GLP,Porto5,Porto5,1
7,Navio457,55,12,47445,9,10,32,17,74,2016,CG,Porto2,Porto2,1
8,Navio208,58,13,95959,16,11,22,15,33,2014,GL,Porto5,Porto5,1
9,Navio168,31,14,123699,18,12,25,17,36,2018,GLP,Porto5,Porto5,1


In [108]:
df_aceitos.to_csv("amostras_aceitos.csv")
df_recusados.to_csv("amostras_recusados.csv")
df_misto.to_csv("amostras_testagem.csv")